In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

import sys
import os

from IPython import display
from time import time
from collections import Counter
import numpy as np
import pandas as pd
from pandas import read_csv
from datetime import datetime
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import matplotlib.pyplot as plt
import seaborn as sns
sns.set() 
sns.set_style('white') 

#!pip install tableone
from tableone import TableOne, load_dataset

import warnings
warnings.filterwarnings('ignore')

print(os.getcwd())

C:\Users\idstwy\Desktop\SSHSPH\SPH6004 Advanced Statistical Learning\Assignment\Group\Codes


In [4]:
#==============
#read data files
#==============
raw = pd.read_csv(r'C:\Users\idstwy\Desktop\SSHSPH\SPH6004 Advanced Statistical Learning\Assignment\Group\Data\SSHSPH_AdvBioStat_AKI\procedureevents.csv.gz', compression='gzip',
                   error_bad_lines=False) #nrows=100,
print(raw.shape)
raw.head(3)

(731247, 26)


,subject_id,hadm_id,stay_id,starttime,endtime,storetime,itemid,value,valueuom,location,locationcategory,orderid,linkorderid,ordercategoryname,secondaryordercategoryname,ordercategorydescription,patientweight,totalamount,totalamountuom,isopenbag,continueinnextdept,cancelreason,statusdescription,comments_date,originalamount,originalrate
0,15693895,21203492,30500789,2147-06-05 11:25:00,2147-06-05 11:26:00,2147-06-06 11:25:00,225399,1.0,None,NaN,NaN,6591564,6591564,Procedures,NaN,Task,80.0,NaN,NaN,0,0,0,FinishedRunning,NaN,1.0,0
1,15693895,21203492,30500789,2147-06-05 22:00:00,2147-06-05 22:01:00,2147-06-06 11:25:00,224385,1.0,None,NaN,NaN,1706064,1706064,Intubation/Extubation,NaN,Task,80.0,NaN,NaN,0,0,0,FinishedRunning,NaN,1.0,0
2,12481680,26876606,30863119,2141-01-23 12:00:00,2141-01-23 18:33:00,2141-01-23 18:33:51.14,224275,393.0,min,NaN,NaN,8373796,8373796,Peripheral Lines,NaN,ContinuousProcess,123.0,NaN,NaN,1,0,0,FinishedRunning,NaN,393.0,1


In [5]:
raw.columns

Index(['subject_id', 'hadm_id', 'stay_id', 'starttime', 'endtime', 'storetime',
       'itemid', 'value', 'valueuom', 'location', 'locationcategory',
       'orderid', 'linkorderid', 'ordercategoryname',
       'secondaryordercategoryname', 'ordercategorydescription',
       'patientweight', 'totalamount', 'totalamountuom', 'isopenbag',
       'continueinnextdept', 'cancelreason', 'statusdescription',
       'comments_date', 'originalamount', 'originalrate'],
      dtype='object')

In [35]:
selectCols=["stay_id", 'starttime', 'endtime', 'orderid',"ordercategoryname"]
raw1 = raw[selectCols]
raw1["Procedure_Intubation_Extubation"] = np.where(raw1["ordercategoryname"] == "Intubation/Extubation",1,0)
raw1["Procedure_Ventilation"] = np.where(raw1["ordercategoryname"] == "Ventilation",1,0)
#raw1.to_csv(r'C:\Users\idstwy\Desktop\SSHSPH\SPH6004 Advanced Statistical Learning\Assignment\Group\Data\SSHSPH_AdvBioStat_AKI\procedure_Intub_Venti.csv', index=False)
raw2 =raw1[(raw1["Procedure_Ventilation"]==1) | (raw1["Procedure_Intubation_Extubation"]==1)]
print(raw1.shape, raw2.shape)

(731247, 7) (67977, 7)


In [36]:
raw2['starttime'] = pd.to_datetime(raw2['starttime'])
raw2['endtime'] = pd.to_datetime(raw2['endtime'])
raw2['duration_hrs'] = (raw2["endtime"] - raw2["starttime"]) / pd.Timedelta(hours=1)
raw2['Procedure_Ventilation'] = np.where(raw2['Procedure_Ventilation']==1,raw2['duration_hrs'],0)
raw2['Procedure_Intubation_Extubation'] = np.where(raw2['Procedure_Intubation_Extubation']==1,raw2['duration_hrs'],0)

raw2['chartDate'] = raw2['starttime'].dt.date
raw2['chartHour'] = raw2['starttime'].dt.hour
raw2 = raw2.drop(columns=['orderid',"ordercategoryname", "endtime"])
raw2 = raw2 [['stay_id', 'starttime', 'chartDate','chartHour', 'Procedure_Intubation_Extubation', 'Procedure_Ventilation']]
raw2 = raw2.sort_values(by=["stay_id", "starttime"])
raw2 = raw2.reset_index(drop=True)
#raw2.to_csv(r'C:\Users\idstwy\Desktop\SSHSPH\SPH6004 Advanced Statistical Learning\Assignment\Group\Data\SSHSPH_AdvBioStat_AKI\procedure_Intub_Venti.csv', index=False)
print(raw2.shape)
raw2.head(3)

(67977, 6)


,stay_id,starttime,chartDate,chartHour,Procedure_Intubation_Extubation,Procedure_Ventilation
0,30000153,2174-09-29 12:00:00,2174-09-29,12,0.000000,5.716667
1,30000153,2174-09-29 17:43:00,2174-09-29,17,0.016667,0.000000
2,30000213,2162-06-21 06:00:00,2162-06-21,6,0.000000,26.900000
